<a href="https://colab.research.google.com/github/hi-rama/AI-Machine-Learning-Study/blob/main/PlantDiseaseDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For this project, we are going to create an end-to-end Android application with TFLite. We opte to develop an Android application that detects plant diseases.

The project is broken down into two steps:
1. Building and creating a machine learning model using TensorFlow with Keras.
2. Deploying the model to an Android application using TFLite. 

# Machine Learning model using Tensorflow with Keras

---



We designed algorithms and models to reconize species and diseases in the crop leaves by using Convolutional Neural Network. We use Colab for edit source code.

Importing the Libraries

In [ ]:
# Install nightly package for some functionalirties that aren't in alpha
!pip install tensorflow-gpu==2.0.0-betal
# Install TF Hub for TF2
!pip install 'tensorflow-hub == 0.5'

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

Loading the data

Download a public dataset of 54,305 images of diseased and healthy plant leaves collected under controlled conditions PlantVillage Dataset. The images cover 14 species of crops, including: apple, blueberry, cherry, grape, orange, peach, pepper, potato, raspberry, soy, squash, strawberry and tomato. It contains images of 17 basic diseases, 4 bacterial diseases, 2 diseases caused by mold(oomycete), 2 viral diseases and 1 disease caused by a mite. 12 crop species also have healthy leaf images that are not visibly affected by disease.

Create the training and validation directories

In [ ]:
#Load data
zip_fil=tf.keras.uils.get_file(origin='https://'fname='PlantVillage.zip',extract=True)
#Create the training and validation directories
data_dir = os.path.join(os.path.dirname(zip_file), 'PlantVillage')
train_dir = os.path.join(data_dir,'train')
validation_dir = os.path.join(data_dir,'validation')

Label mapping

You'll also need to load in a mapping from category label to category name. This will give you a dictionary mapping the integer encoded categories to the actual names of the plants and diseases.

In [ ]:
!wget https://github.com/obeshor/Plant-diseases-Detector/archive/master.zip
!unzip master.zip;
import json
with open('Plant-Diseases-Detector-master/categories.json','r') as f:
  cat_to_name = json.load(f)
  classes = list(cat_to_name.values())
print(classes)

Transfer Learning with TensorFlow Hub

Select the Hub/TF2 module to use, you have a choice with inception v3 or Mobilenet

In [ ]:
module_selection = ("inception_v3", 299, 2048) #@param ["(\"mobilenet_v2\", 224, 1280)", "(\"inception_v3\", 299, 2048)"] {type:"raw", allow-input: true}
handle_base,pixels,FV_SIZE = module_selection
MODULE_HANDLE = "https://tfhub.dev/google/tf2-preview/{}/feature_vector/2".format(handle_base)
IMAGE_SIZE = (pixels,pixels)
BATCH_SIZE = 64 #@param{type:"integer"}

Data Preprocessing

Let's set up data generators that will read pictures in our source folders, convert them to 'float32' tensors, and feed them to our network

As you may already know, data that goes into neural networks should usually be normanized in some way to make it more amenable to processing by the network. In our case, we will preprocess our images by normalizing the pixel values to be in the '[0,1]' range (originally all values are in the '[0,255]' range). We'll need to make sure the input data is resized to 224x224 pixels or 299x299 pixels as required by the networks. You have the choice to implement image augmentation or not.

In [ ]:
# Input are suitably resized for the selected module.
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./225)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    shuffle=False,
    seed=42,
    color_mode="rgb"
    class_mode="categorical",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)
do_data_augmentation = True #@param (type:"boolean")
if do_data_augmentation:
  train_datagen = tf.keras.preprocessing.imgae.ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      horizontal_flip=True,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      fill_mode='nearest'
  )
else:
  train_datagen = validation_datagen
train_generator = train_datagen.flow_from_directory(
    train_dir,
    subset="training",
    shuffle=True,
    seed=42,
    color_mode="rgb",
    class_mode="categorical",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)
)

Build the model

All it takes is to put a linear classifier on top of the feature_extractor with the Hub module. For speed, we start out with  a non-trainable feature_extractor, but you can also enable fine-tuning for greater accuracy but that taks a lot of time to train model

In [ ]:
featur_extractor = hub.KerasLayer(MODULE_HANDLE,
                                  input_shape=IMAGE_SIZE+(3,),
                                  output_shape=[FV_SIZE])
do_fine_tuning = False #@param {type:"boolean"}
if do_fine_tuning:
  feature_extractor.trainable = True
  # unfreeze some layers of base network for fine-tuning
  for layer in feature_extractor.layers[-30]:
    layer.trainable =True
  else:
    feature_extractor.tarinable = False
model = tf.keras.Sequential([
      feature_extractor,
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512,activation="relu")
      tf.keras.layers.Dropout(rate=0.2),
      tf.keras.layers.Dense(train_generator.num_classes, activation='softmax',
                            kernel_regularizer=tf.keras.regularizers.12(0.0001))
])

Specifying Loss Function and Optimizer

In [ ]:
#Compile model specifying the optimizer learning rate
LEARNING_RATE = 0.001 #@param {type:"number"}
model.compile(
    optimizer=tf.keras.opimizers.Adam(lr=LEARNING_RATE),
    loss='categorical_cressentropy',
    metrics=['accuracy']
)

Training Model

train model using validation dataset for validate each steps. After 10 epochs, we get 94% for accuracy, you can improve this more than 99% using fine-tuning

In [ ]:
EPOCES=10 #@param {type:"integer"}
STEPS_EPOCHES= train_generator.samples//train_generator.batch_size
VALID_STEPS=validation_generator.samples//validation_generator.batch_size
histroy = model.fit_generator(
    tran_generator,
    steps_per_epoch = STEPS_EPOCHS,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=VALID_STEPS
)

Checking Performance

Plot training and validation, accuracy and loss

In [ ]:
import matplotlib.pylab as plt
import numpy as np
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss= history.history['loss']
val_loss=history.history['val_loss']
epochs_range = range(EPOCHS)
plt.figure(figsize=(8,8))
plt.subplt(1,2,1)
plt.plot(epochs_range,acc,label='Trainig Accuracy')
plt.plot(epochs_range,val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")
plt.subplot(1,2,2)
plt.plot(epochs_range,loss,label='Training Loss')
plt.plot(epochs_range,val_loss, label='Validaion Loss')
plt.legend(loc='upper right')
plt.title('Training and validation loss')
plt.ylabel('loss(training and validation')
plt.xlable("Training steps")
plt.show()

NameError: ignored

Random test

Random five sample images from validation dataset and predict:

In [ ]:
#import OpenCV
import cv2
#Utility
import itertools
import random
from collection import Counter
from golb import iglob
def load_image(filename):
  img = cv2.imread(os.pah.join(data_dir, validation_dir, filename))
  img = cv2.resize(img, (IMAGE_SIZE[0],IMAGE_SIZE[1]))
  img = img/255
  return img
def predic(image):
  probabilities = model.predict(np.asarray([img]))[0]
  class_idx = np.argmax(probabilities)
  return {classes[class_idx]: probabilities[class_idx]}
for idx, filename in enumerate(random.sample(validation_generator.filename,5)):
  print("SOURCE: class: %s, file: %s" % (os.path.split(filename)[0],filename))
  img = load_image(filename)
  prediction = predict(img)
  print("PREDICTED: class: %s, confidene: %f" % (list(prediction.keys())[0], list(prediction.values())[0]))
  plt.imshow(img)
  plt.figure(idx)
  plt.show()

The model can be improved if you change some hyperparameters. You can try using a different pretrained model. It's up to you. Let me know if you can improve the accuracy!

Convert model to TensorFlow Lite

In [ ]:
#convert the model to TFLite
!mkdir "tflite_models"
TFLITE_MODEL = "tflite_models/plant_disease_model.tflite"

#Get the concrete function from the Keras model.
run_model = tf.function(lambda x : reloaded(x))

#Save the concrete function
concrete_func = run_model.get_concreter_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype)
)
#Convert the model to standard TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converted_tflite_model = converter.convert()
open(TFLITE_MODEL, "wb").write(converted_tflite_model)


Add TFLite model in our Android Project

First-load the model in our Androi project, we put plant_disease_model.tflite and plant_labels.txt into assets/ directory. plant_disease_model.tflite is the result of our previous colab notebook. We need to add TFLite dependency to app/build.gradle file.


In [ ]:
aaptOptions {
    noCompress "tflite"
}

Dive into the code

create classifier class to load our model and read the file with labels:

In [ ]:
from io import BufferedReader
from google.protobuf.descriptor import FileDescriptor
from tensorflow.lite.python.interpreter import Interpreter
class Classifier(assetManager: AssetManager, modelPath: String, labelPath: String, inputSize: int){
    private var Interpreter: Interpreter
    private var LABEL_LIST: List<String>
    private val INPUT_SIZE: Int = inputSize
    private val PIXEL_SIZE: Int = 3
    private val IMAGE_MEAN = 0
    private val IMAGE_STD = 255.0f
    private val MAX_RESULTS = 3
    private val THRESHOLD = 0.4f

    ...
    init{
        INTERPRETER = Interpreter(loadModelFile(assetManager, modelPath))
        LABEL_LIST = loadLabelList(assetManager,labelPath)
    }
    private fun loadModelFile(assetManager: AssetManager, modelPath: String):MappedByteBuffer{
        val FileDescriptor = assetManager.openFd(modelPath)
        val inputStream = FileInputStream(fileDescriptor.fileDescriptor)
        val fileChannel = inputStream.channel
        val startOffset = FileDescriptor.startOffset
        val declaredLength = filDescriptor.declaredLength
        return fileChannel.map(FileChannel.MapMode.READ_ONLY,startOffset,declaredLength)
    }
    private fun loadLabelList(assetManager: AssetManager, labelPath:String):List<String>{
        return assetManager.open(labelPath).BufferedReader().useLines{it.toList()}
    }
...
}

Where Recognition is our humble result data class:

In [ ]:
data class Recognition(
        var id: String = "",
        var title: String = "",
        var confidence: Float = 0F
    )  {
        override fun toString(): String {
            return "Title = $title, Confidence = $confidence)"
        }
    }

When we have an instance of Interpreter, we need to convert the preprocessed bitmap into ByteBuffer then we create a method that will take an image as an argument and return a list of labels with assigned probabilities to them:

In [ ]:
fun recognizeImage(bitmap: Bitmap): List<Classifier.Recognition>{
    val scaledBitmap = Bitmap.createScaleBitmap(bitmap, INPUT_SIZE, INPUT_SIZE, false)
    val byteBuffer = convertBitmapToByteBuffer(scaledBitmap)
    val result = Array(1){FloatArray(LABEL_LIST.size)}
    INTERPRETER.run(byteBuffer, result)
    return getSortedResult(result)
}

Here's how we convert a bitmap into ByteBuffer:

In [ ]:
private fun convertBitmapToByteBuffer(bitmap: Bitmap):ByteBuffer {
    val byteBuffer = ByterBuffer.allocaeDirect(4*INPUT_SIZE*INPUT_SIZE*PIXEL_SIZE)
    byteBuffer.order(ByteOrder.nativeOrder())
    val intValues = IntArray(INPUT_SIZE*INPUT_SIZE)
    bitmap.getPixels(intValues,0,bitmap.width,0,0,bitmap.width,bitmap.height)
    var pixel = 0
    for(i in 0 until INPUT_SIZE){
        for(j in 0 until INPUT_SIZE)p
        val 'val' = intValues[pixel++]
        byteBuffer.putFloat(((('val'.shr(16) and 0xFF) - IMAGE_MEAN / IMAGE_STD))))
        byteBuffer.putFloat(((('val'.shr(+) and 0xFF) - IMAGE_MEAN / IMAGE_STD))))

        byteBuffer.putFloat(((('val'.shr(16) and 0xFF) - IMAGE_MEAN / IMAGE_STD))

    }
}
return byteBuffer